# Práctica Final: Clasificación con Scikit-learn y MLflow

En esta práctica, utilizarás un conjunto de datos de Scikit-learn (podeís usar el mismo que en el notebook de Intro MLFlow) para entrenar un modelo de clasificación.

Pasos a seguir: 

    Exploración de Datos: Analiza el conjunto de datos proporcionado para comprender su estructura y contenido.

    Preprocesamiento de Texto: Realiza tareas de preprocesamiento de texto, como tokenización y vectorización, para preparar los datos para el modelado.

    Entrenamiento del Modelo: Utiliza algoritmos de clasificación de Scikit-learn para entrenar un modelo con los datos preprocesados.

    Evaluación del Modelo: Evalúa el rendimiento del modelo utilizando métricas de evaluación estándar como precisión y recall.

    Registro de Métricas con MLflow: Utiliza MLflow para registrar métricas y hiperparámetros durante el entrenamiento, facilitando la gestión y comparación de experimentos.


Nota: Dado que no voy a poder tener acceso a vuestros logs de MLFlow añadirme las imagenes de la interfaz de MLFlow en el notebook

In [1]:
# importamos las librerias necesarias
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# vamos a utilizar un datasets de vinos de un análisis químico de vinos cultivados en la misma región de Italia por tres cultivadores diferentes
from sklearn.datasets import load_wine

## EXPLORACION DE LOS DATOS

In [2]:
# cargamos el dataset
vinos = load_wine()

# lo pasamos a un dataframe
df_vinos = pd.DataFrame(vinos['data'], columns=vinos['feature_names'])

In [3]:
# observamos los datos 
df_vinos.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [4]:
# verificamos que no haya campos vacios y sean todos valores numericos
print(df_vinos.isnull().value_counts())

print(f'Shape del df {df_vinos.shape}')

alcohol  malic_acid  ash    alcalinity_of_ash  magnesium  total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity  hue    od280/od315_of_diluted_wines  proline
False    False       False  False              False      False          False       False                 False            False            False  False                         False      178
Name: count, dtype: int64
Shape del df (178, 13)


In [5]:
df_vinos.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [6]:
# vemos las columnas
df_vinos.columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'],
      dtype='object')

## PREPROCESAMIENTO DE DATOS
Al ser un datasets bajado de scikit-learn no hace falta preprocesar mucha informacion de los datos. Vamos a dividir los datos y a normalizarlos

In [7]:
# añadimos la columna de las etiquetas al dataframe
df_vinos['target'] = vinos['target']

df_vinos.head(100)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,12.47,1.52,2.20,19.0,162.0,2.50,2.27,0.32,3.28,2.60,1.16,2.63,937.0,1
96,11.81,2.12,2.74,21.5,134.0,1.60,0.99,0.14,1.56,2.50,0.95,2.26,625.0,1
97,12.29,1.41,1.98,16.0,85.0,2.55,2.50,0.29,1.77,2.90,1.23,2.74,428.0,1
98,12.37,1.07,2.10,18.5,88.0,3.52,3.75,0.24,1.95,4.50,1.04,2.77,660.0,1


In [8]:
# dividimos los datos un 25% para test
train, test = train_test_split(df_vinos,test_size=0.25) 

# eliminamos de test la variable target para luego poder usarlo 
features = [x for x in list(test.columns) if x != 'target']
x_test = test[features]
y_test = test['target']

# obtenemos la columna target de train para luego volver a dividirlo entre datos de entrenamiento y de validacion 
features = [x for x in list(train.columns) if x != 'target']
x_train = train[features]
y_train = train['target']



In [9]:
# observamos que la proporcion de datos es correcta
print(y_train.value_counts())
print(y_test.value_counts())
print(x_train.shape, x_test.shape)

target
1    54
0    44
2    35
Name: count, dtype: int64
target
1    17
0    15
2    13
Name: count, dtype: int64
(133, 13) (45, 13)


In [10]:
# normalizamos los datos 
normalizacion = Pipeline(steps=[('scaler', StandardScaler())])

## ENTRENAMIENTO DEL MODELO
Para el entramiento vamos a utilizar el modelo GradientBoostingClassifier 

In [11]:
# creamos el modelo y luego lo añadimos al pipeline

gbc = GradientBoostingClassifier(
    n_estimators=100,      
    learning_rate=0.1,     
    max_depth=3,           
    random_state=19
)

model = Pipeline(steps=[('preprocessor', normalizacion),
                           ('classifier', gbc)])


In [12]:
# entrenamos el modelo 
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('scaler', StandardScaler())])),
                ('classifier', GradientBoostingClassifier(random_state=19))])

## Evaluacion del modelo

In [13]:
# predecimos los datos de test
y_pred = model.predict(x_test)

In [14]:
accuracy_train = model.score(x_train, y_train)
print(f"Accuracy del modelo en train: {accuracy_train:.2f}")

Accuracy del modelo en train: 1.00


In [15]:
# calculamos el accuracy 
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del modelo en test: {accuracy:.2f}")

print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred))

# metricas = classification_report(y_test, y_pred, output_dict=True)


Accuracy del modelo en test: 0.96

Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       1.00      0.88      0.94        17
           2       0.93      1.00      0.96        13

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.95        45



In [23]:
metricas = classification_report(y_test, y_pred, output_dict=True)

# metricas['1']['precision']
# metricas['1']['f1-score']
metricas

{'0': {'precision': 0.9375,
  'recall': 1.0,
  'f1-score': 0.967741935483871,
  'support': 15.0},
 '1': {'precision': 1.0,
  'recall': 0.8823529411764706,
  'f1-score': 0.9375,
  'support': 17.0},
 '2': {'precision': 0.9285714285714286,
  'recall': 1.0,
  'f1-score': 0.9629629629629629,
  'support': 13.0},
 'accuracy': 0.9555555555555556,
 'macro avg': {'precision': 0.9553571428571429,
  'recall': 0.9607843137254902,
  'f1-score': 0.9560682994822779,
  'support': 45.0},
 'weighted avg': {'precision': 0.958531746031746,
  'recall': 0.9555555555555556,
  'f1-score': 0.9549366122394797,
  'support': 45.0}}

## Registro de metricas
Primero añadiremos todo lo realizado en una celda y añadiremos mlflow con los argumentos y metricas que capturamos

In [25]:
# importamos las librerias necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.datasets import load_wine

vinos = load_wine()
df_vinos = pd.DataFrame(vinos['data'], columns=vinos['feature_names'])
df_vinos['target'] = vinos['target']

# dividimos los datos
train, test = train_test_split(df_vinos,test_size=0.25) 

features = [x for x in list(test.columns) if x != 'target']
x_test = test[features]
y_test = test['target']

features = [x for x in list(train.columns) if x != 'target']
x_train = train[features]
y_train = train['target']

# hiperparametros
n_stimators = [30,50,80,100,130]
max_depth = [3,5,8]

mlflow.set_experiment('Modelo vinos')
for i in n_stimators:
  for l in max_depth:
    with mlflow.start_run(run_name=f'N_estimador{i} - max_depth{l}') as run:
      run_num = run.info.run_id
      model_uri = 'runs:/{run_id}/artifact_path'.format(run_id=run_num, artifact_path='modelwml')
      gbc = GradientBoostingClassifier(
          n_estimators=i,      
          learning_rate=0.1,     
          max_depth=l,           
          random_state=19
      )

      normalizacion = Pipeline(steps=[('scaler', StandardScaler())])

      model = Pipeline(steps=[('preprocessor', normalizacion),
                                ('classifier', gbc)])

      model.fit(x_train, y_train)

      accuracy_train = model.score(x_train, y_train)
      accuracy_test =model.score(x_test, y_test)
      y_pred = model.predict(x_test)

      # obtenemos las metricas en un diccionario
      metricas = classification_report(y_test, y_pred, output_dict=True)

      # metricas clase 0
      mlflow.log_metric('precision_clase0', metricas['0']['precision'])
      mlflow.log_metric('recall_clase0', metricas['0']['recall'])
      mlflow.log_metric('f1-score_clase0', metricas['0']['f1-score'])

      # metricas clase 1
      mlflow.log_metric('precision_clase1', metricas['1']['precision'])
      mlflow.log_metric('recall_clase1', metricas['1']['recall'])
      mlflow.log_metric('f1-score_clase1', metricas['1']['f1-score'])

      # metricas clase 2
      mlflow.log_metric('precision_clase2', metricas['2']['precision'])
      mlflow.log_metric('recall_clase2', metricas['2']['recall'])
      mlflow.log_metric('f1-score_clase2', metricas['2']['f1-score'])

      mlflow.log_metric('accuraty_train', accuracy_train)
      mlflow.log_metric('accuracy_test', accuracy_test)
      mlflow.log_param('n_stimators', i)
      mlflow.log_param('max_depth', l)
      mlflow.sklearn.log_model(model, 'despliegue-algoritmos-model')
      model_details = mlflow.register_model(
          model_uri=model_uri,
          name='modelwml'
      )



2024/12/15 15:58:45 INFO mlflow.tracking.fluent: Experiment with name 'Modelo vinos' does not exist. Creating a new experiment.
2024/12/15 15:58:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'modelwml'.
Created version '1' of model 'modelwml'.
2024/12/15 15:58:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'modelwml' already exists. Creating a new version of this model...
Created version '2' of model 'modelwml'.
2024/12/15 15:58:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'modelwml' already exists. Creating a new version of this m

## Capturas Mlflow


<img src="./imagenes/comparacionModels.PNG" alt="Mlflow"/>
<img src="./imagenes/comparacionmodels2.PNG" alt="Mlflow" />
<img src="./imagenes/experimentogeneral.png" alt="Mlflow" />
<img src="./imagenes/modelregistri.PNG" alt="Mlflow" />
<img src="./imagenes/artifacts.png" alt="Mlflow" />

## Generar .py de funciones y main con al menos dos argumentos de entrada.

In [27]:
# funciones.py 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.datasets import load_wine

def argumentos():
    parser = argparse.ArgumentParser(description='__main__ de la aplicación con argumentos de entrada.')
    parser.add_argument('--nombre_job', type=str, help='Valor para el parámetro nombre_documento.')
    parser.add_argument('--n_estimators_list', nargs='+', type=int, help='List of n_estimators values.')
    parser.add_argument('--max_depth_list', nargs='+', type=int, help='List of n_estimators values.')
    return parser.parse_args()

def load_dataset():
    vinos = load_wine()
    df_vinos = pd.DataFrame(vinos['data'], columns=vinos['feature_names'])
    df_vinos['target'] = vinos['target']
    return df_vinos

def data_treatment(df):
    # Split data into train and test sets
    train, test = train_test_split(df,test_size=0.25,random_state=19) 

    features = [x for x in list(test.columns) if x != 'target']
    x_test = test[features]
    y_test = test['target']

    features = [x for x in list(train.columns) if x != 'target']
    x_train = train[features]
    y_train = train['target']

    return x_train, x_test, y_train, y_test

def mlflow_tracking(nombre_job, x_train, x_test, y_train, y_test, n_estimators,max_depths):
    mlflow.set_experiment(nombre_job)
    for i in n_estimators:
        for l in max_depths:
            with mlflow.start_run(run_name=f'N_estimador{i} - max_depth{l}') as run:
                gbc = GradientBoostingClassifier(
                    n_estimators=i,      
                    learning_rate=0.1,     
                    max_depth=l,           
                    random_state=19
                )

                normalizacion = Pipeline(steps=[('scaler', StandardScaler())])

                model = Pipeline(steps=[('preprocessor', normalizacion),
                                            ('classifier', gbc)])

                model.fit(x_train, y_train)

                accuracy_train = model.score(x_train, y_train)
                accuracy_test =model.score(x_test, y_test)
                y_pred = model.predict(x_test)

                # obtenemos las metricas en un diccionario
                metricas = classification_report(y_test, y_pred, output_dict=True)

                # metricas clase 0
                mlflow.log_metric('precision_clase0', metricas['0']['precision'])
                mlflow.log_metric('recall_clase0', metricas['0']['recall'])
                mlflow.log_metric('f1-score_clase0', metricas['0']['f1-score'])

                # metricas clase 1
                mlflow.log_metric('precision_clase1', metricas['1']['precision'])
                mlflow.log_metric('recall_clase1', metricas['1']['recall'])
                mlflow.log_metric('f1-score_clase1', metricas['1']['f1-score'])

                # metricas clase 2
                mlflow.log_metric('precision_clase2', metricas['2']['precision'])
                mlflow.log_metric('recall_clase2', metricas['2']['recall'])
                mlflow.log_metric('f1-score_clase2', metricas['2']['f1-score'])

                mlflow.log_metric('accuraty_train', accuracy_train)
                mlflow.log_metric('accuracy_test', accuracy_test)
                mlflow.log_param('n_stimators', i)
                mlflow.log_param('max_depth', l)
                mlflow.sklearn.log_model(model, 'despliegue-algoritmos-model')

    print("Se ha acabado el entrenamiento del modelo correctamente")



UsageError: Line magic function `%%writefile` not found.


In [28]:
# main.py
from funciones import argumentos, load_dataset, model, mlflow_tracking

def main():
  print("Eejcutamos el main")
  args_values = argumentos()
  df = load_dataset()
  x_train, x_test, y_train, y_test = data_treatment(df)
  mlflow_tracking(args_values.nombre_job, x_train, x_test, y_train, y_test, args_values.n_estimators_list,args_values.max_depth_list)

if __name__ == "__main__":
  main()


ModuleNotFoundError: No module named 'funciones'

## Práctica parte FastAPI

### Para esta parte de la práctica teneis que generar un script con al menos 5 modulos app.get y dos de ellos tienen que ser pipelines de HF. 

### Parte de la practica se tendra que entregar en capturas de pantalla. Las capturas de pantalla a adjuntas son las siguientes. 

### 1. Captura de la pantalla docs con al menos 5 modulos. 
### 2. Captura de cada una de los modulos con la respuesta dentro de docs. 
### 3. Captura de cada uno de los modulos en la llamada https.
### 4. Todo el codigo usado durante el proceso. Notebooks y scripts.

### Opcional

### 5. Despliegue del script en GCP Cloud Run

#### 1. Captura de la pantalla docs con al menos 5 modulos. 

<img src="./imagenes/fastapi1.PNG" alt="Fastapi"/>

#### 2. Captura de cada una de los modulos con la respuesta dentro de docs.

##### Clasificacion de imagenes 
<img src="./imagenes/fastapi2-1.PNG" alt="Fastapi"/>


##### Traductor
<img src="./imagenes/fastapi2-2.PNG" alt="Fastapi"/>

##### Analisis de sentiemiento
<img src="./imagenes/fastapi2-3.PNG" alt="Fastapi"/>

##### Clasificador de coches

<img src="./imagenes/fastapi2-4.PNG" alt="Fastapi"/>

##### De imagen a texto


<img src="./imagenes/fastapi2-5.PNG" alt="Fastapi"/>

##### Imagenes de prueba 

<img src="./imagenes/mercedes.jpg" alt="Fastapi" width="500"/>
<img src="./imagenes/pug.png" alt="Fastapi" width="500"/>

#### 3. Captura de cada uno de los modulos en la llamada https.

In [43]:
import requests

def metodosGet(tipoModelo, text):
    
    texto = {
        "text": text
    }

    url = 'http://127.0.0.1:8000/'+tipoModelo

    # realizamos la peticion 
    response = requests.get(url, params=texto)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    

print(f'Respuesta del modelo de analisis de sentimiento: {metodosGet("analisisSentimiento", "es mediocre ese producto")}')
print(f'Respuesta del modelo de traducción: {metodosGet("traduccion", "this product is bad")}')

Respuesta del modelo de analisis de sentimiento: [{'label': '2 stars', 'score': 0.5410885810852051}]
Respuesta del modelo de traducción: este producto es malo


In [50]:
import requests

def metodosPost(tipoModelo, rutaImagen):

    url = 'http://127.0.0.1:8000/'+tipoModelo

    # abrimos la imagen con los permisos necesarios
    with open(rutaImagen, "rb") as image_file:
        files = {"file": image_file}  
        response = requests.post(url, files=files)


    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")


print(f"Respuesta modelo de imagen a texto con una imagen de un perro: {metodosPost('imagenToTexto', './pug.png')}")
print('-------------------')
print(f"Respuesta modelo de clasificador de coches: { metodosPost('clasificarCoches', './mercedes.jpg') }")
print('-------------------')
print(f"Respuesta modelo de clasificador de imagenes: { metodosPost('clasificadorImagenes', './pug.png') }")

Respuesta modelo de imagen a texto con una imagen de un perro: Pug cachorro sentado en el suelo
-------------------
Respuesta modelo de clasificador de coches: [{'label': 'SUV', 'score': 0.8753702044487}, {'label': 'Hatch-back', 'score': 0.4619683623313904}, {'label': 'Pick-up Truck', 'score': 0.42045965790748596}, {'label': 'Sedan', 'score': 0.404720276594162}, {'label': 'VAN', 'score': 0.25464022159576416}]
-------------------
Respuesta modelo de clasificador de imagenes: [{'label': 'pug, pug-dog', 'score': 0.9560531377792358}, {'label': 'bull mastiff', 'score': 0.023975420743227005}, {'label': 'Brabancon griffon', 'score': 0.006916860118508339}, {'label': 'French bulldog', 'score': 0.001122591900639236}, {'label': 'Labrador retriever', 'score': 0.0008803399978205562}]


#### Todo el codigo usado durante el proceso. Notebooks y scripts.

<img src="./imagenes/fastapi4.PNG" alt="Fastapi"/>